In [5]:
import gym
import getch
from stable_baselines3.common.env_checker import check_env
from bombergym.scenarios import classic, classic_with_opponents, coin_heaven
from bombergym.environments import register
import torch
import time
import numpy as np

np.set_printoptions(linewidth=120)

register()
settings, agents = classic_with_opponents()

env = gym.make('BomberGym-v0', args=settings, agents=agents)

In [18]:
import gmpy2
gmpy2.digits(10, 11)

'a'

In [23]:
%%timeit
gmpy2.digits(10, 11)

175 ns ± 0.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [27]:
int('AAAAAA', base=11)

1771560

In [7]:
EXPLOSION = -4
BOMB_T0 = -3
BOMB_T1 = -2
BOMB_T2 = -1
BOMB_T3 = 0
WALL = 1
ENEMY = 2
SELF = 3
FREE = 4
CRATE = 5
COIN = 6

In [6]:
env.reset()

array([[-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
       [-2,  2,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  0, -1, -2],
       [-2,  0, -2,  0, -2,  1, -2,  0, -2,  0, -2,  1, -2,  1, -2,  0, -2],
       [-2,  1,  1,  0,  1,  1,  0,  1,  0,  0,  0,  1,  1,  0,  1,  1, -2],
       [-2,  0, -2,  1, -2,  1, -2,  1, -2,  1, -2,  0, -2,  0, -2,  1, -2],
       [-2,  1,  1,  1,  1,  0,  1,  1,  0,  1,  1,  0,  1,  1,  0,  1, -2],
       [-2,  1, -2,  1, -2,  1, -2,  1, -2,  1, -2,  1, -2,  1, -2,  1, -2],
       [-2,  1,  1,  1,  0,  0,  1,  0,  1,  0,  1,  1,  1,  0,  0,  1, -2],
       [-2,  1, -2,  1, -2,  0, -2,  1, -2,  0, -2,  1, -2,  1, -2,  1, -2],
       [-2,  1,  1,  1,  0,  1,  0,  0,  1,  0,  1,  1,  0,  0,  0,  1, -2],
       [-2,  1, -2,  1, -2,  1, -2,  1, -2,  0, -2,  1, -2,  1, -2,  0, -2],
       [-2,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1,  1,  1,  0,  0,  1, -2],
       [-2,  1, -2,  1, -2,  0, -2,  1, -2,  1, -2,  1, -2,  1, -2,  1, -2],

In [ ]:


def detect_initial_configuration(obs):
    agent_frame = obs[4, :, :]
    x, y = (agent_frame == 1).nonzero()
    x, y = x[0], y[0]
    config = None
    if x < 7 and y > 7:
        config = 'bottom-left'
    if x > 7 and y < 7:
        config = 'top-right'
    if x < 7 and y < 7:
        config = 'top-left'
    if x > 7 and y > 7:
        config = 'bottom-right'
    return config

def get_transposer(config):
    if config == 'top-left':
        return lambda model, input: model(torch.from_numpy(input).unsqueeze(0)).argmax().item()
    elif config == 'top-right':
        return transposer_lr
    elif config == 'bottom-left':
        return transposer_td
    else:
        return transposer_lrtd

def transposer_lr(model, inp):
    input_aug = inp[:, ::-1, :].copy()
    action = model(torch.from_numpy(input_aug).unsqueeze(0)).argmax().item()
    new_action_lr = None
    if action == 1: # Right
        new_action_lr = 3 # Left
    elif action == 3: # Left
        new_action_lr = 1 # Right
    else:
        new_action_lr = action
    return new_action_lr

def transposer_td(model, inp):
    input_aug = inp[:, :, ::-1].copy()
    action = model(torch.from_numpy(input_aug).unsqueeze(0)).argmax().item()
    new_action_ud = None
    if action == 0: # Up
        new_action_ud = 2 # down
    elif action == 2: # Down
        new_action_ud = 0 # Up
    else:
        new_action_ud = action
    return new_action_ud

def transposer_lrtd(model, inp):
    input_aug = inp[:, ::-1, ::-1].copy()
    action = model(torch.from_numpy(input_aug).unsqueeze(0)).argmax().item()
    new_action_udlr = None
    if action == 0: # Up
        new_action_udlr = 2 # down
    elif action == 2: # Down
        new_action_udlr = 0 # Up
    elif action == 1: # Right
        new_action_udlr = 3 # Left
    elif action == 3: # Left
        new_action_udlr = 1 # Right
    else:
        new_action_udlr = action
    return new_action_udlr

obs = env.reset()
previous_obs = None
initial_config = detect_initial_configuration(obs)
transposer = get_transposer(initial_config)
print(f'Config: {initial_config}')
time.sleep(1)
env.render()
while True:
    if previous_obs is not None:
        concat_obs = np.dstack((previous_obs, obs)).astype(np.float32)
        print(model(torch.from_numpy(concat_obs).unsqueeze(0)))
        action = model(torch.from_numpy(concat_obs).unsqueeze(0)).argmax().item()
        #action = transposer(model, concat_obs)
    else:
        action = 2
    previous_obs = obs
    obs, rew, done, other = env.step(action)
    if not done:
        feature_info = other["features"] if "features" in other else None
        env.render(events=other["events"], rewards=rew, other=feature_info)
        time.sleep(.5)
    else:
        print(other["events"], f"Reward: {rew}")
        break
    
 
